# Objects

In [1]:
%load_ext autoreload
%autoreload 2

import crispr as cr
from anndata import AnnData
import scanpy as sc
import copy

harmony = True
kws_process_guide_rna = {"feature_split": "|", "guide_split": "-", 
                         "key_control_patterns": ["CTRL"]}
kws_umap = {"min_dist": 0.3}
kws_umap=kws_umap
kws_init = {"assay": None, "assay_protein": None, 
            "col_gene_symbols": "gene_symbols", 
            "col_cell_type": "majority_voting", 
            "col_sample_id": None, "col_batch": None, 
            "col_perturbed": "perturbation", 
            "col_guide_rna": "feature_call", 
            "col_num_umis": "num_umis", 
            "kws_process_guide_rna": kws_process_guide_rna,
            "col_condition": "target_gene_name", "key_control": "NT", 
            "key_treatment": "KD", "remove_multi_transfected": True}
kws_pp = {"kws_hvg": {"min_mean": 0.0125, "max_mean": 10, 
                      "min_disp": 0.5, "flavor": "cell_ranger"}, 
          "target_sum": 10000.0, "cell_filter_pmt": [0, 15], 
          "cell_filter_ngene": [200, None], "cell_filter_ncounts": [3, 40000], 
          "gene_filter_ncell": [3, None], "regress_out": None, 
          "kws_scale": "z", "kws_umap": kws_umap}
kws_cluster = {"method_cluster": "leiden", "kws_umap": kws_umap, 
               "kws_neighbors": None, 
               "kws_pca": {"n_comps": None, "use_highly_variable": True}, 
               "kws_cluster": {"resolution": 0.5}}

file_path = {'HH03': {'directory': 'data/crispr-screening/HH06'}, 
             'CR4': {'directory': 'data/crispr-screening/HH-Hu-CR4'},
             'CR5': {'directory': 'data/crispr-screening/HH-Hu-CR5'}}



# Create Integrated Object

In [3]:
adata = cr.pp.create_object_multi(file_path, kws_init, kws_pp, kws_cluster, 
                                  harmony=harmony)  # create AnnData object
kws_init_new = copy.deepcopy(kws_init)
kws_init_new["kws_process_guide_rna"] = None  # don't need any more
self = cr.Crispr(adata, **kws_init)  # pass integrated object to Crispr class
del(adata)  # object now in `self`



<<< INITIALIZING CRISPR CLASS OBJECT >>>


<<< LOADING PROTOSPACER METADATA >>>


# Clustering & Annotations

Clustering was conducted in the individual objects in `cr.pp.create_object_multi()`, but we want to re-do QC and clustering in the overall sample.

## Clustering

In [ ]:
self.cluster(**kws_cluster)
degs = self.find_markers(n_genes=10, method="wilcoxon", key_reference="rest")
degs[0]  # marker data; degs[1] for figures

## CellTypist

Annotate cell types.

In [ ]:
preds, f_ct = self.annotate_clusters(model="COVID19_HumanChallenge_Blood.pkl")
self.rna.obs

# Plots

In [ ]:
cct = "predicted_labels" if "predicted_labels" in self.rna.obs else None
clus = list(self.rna.obs[cct].sample(2))  # clusters to circle
figs = self.plot(genes=36, 
                 col_cell_type=cct,
                 layers="all",  # also plot "scaled" layer, for instance
                 kws_gex_violin=dict(scale="area", height=10),
                 kws_clustering=dict(col_cell_type=cct))

# Analyses

Perform perturbation-specific analyses.

## Mixscape

In [ ]:
figs_mixscape = self.run_mixscape(
    col_split_by=self._columns["col_sample_id"], col_cell_type=cct, 
    target_gene_idents=True)

## Augur

figs_mixscape = self.run_mixscape(
    col_split_by=self._columns["col_sample_id"], col_cell_type=cct, 
    target_gene_idents=True)

# Distance

Investigate distance metrics.